# TSA Chapter 4: Why SARIMA Is Needed

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch4/TSA_ch4_motivation_why_sarima/TSA_ch4_motivation_why_sarima.ipynb)

ARIMA residuals still contain seasonal autocorrelation.

In [ ]:
!pip install numpy pandas matplotlib statsmodels -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Load airline passengers
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv',
                    parse_dates=['Month'], index_col='Month')
passengers = data['Passengers']
passengers.index.freq = 'MS'
log_passengers = np.log(passengers)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
acf_orig = acf(passengers, nlags=48)
colors = [RED if i > 0 and i % 12 == 0 else BLUE for i in range(len(acf_orig))]
axes[0].bar(range(len(acf_orig)), acf_orig, color=colors, width=0.5)
axes[0].set_title('ACF: Seasonal Spikes at 12, 24, 36', fontweight='bold')
try:
    m = SARIMAX(log_passengers, order=(1,1,1), seasonal_order=(0,0,0,0)); r = m.fit(disp=False)
    acf_res = acf(r.resid[2:].dropna(), nlags=36)
except: acf_res = np.zeros(37); acf_res[0]=1; acf_res[12]=0.45
cols2 = [RED if i > 0 and i % 12 == 0 else ORANGE for i in range(len(acf_res))]
axes[1].bar(range(len(acf_res)), acf_res, color=cols2, width=0.5)
axes[1].set_title('ARIMA(1,1,1) Residuals: Seasonality Remains', fontweight='bold')
plt.tight_layout(); save_chart(fig, 'ch4_motivation_why_sarima'); plt.show()